In [1]:
# Path setting
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/LDO/爬蟲')
os.listdir()

Mounted at /content/drive


['vegemap_data.ipynb']

In [2]:
!pip install requests beautifulsoup4


In [4]:
import requests
from bs4 import BeautifulSoup

# 要爬的網站 URL（請更換成你的實際網址）
url = "https://vegemap.merit-times.com/restaurant_list/%E5%9F%BA%E9%9A%86%E5%B8%82-%E8%94%AC%E9%A3%9F%E9%A4%90%E5%BB%B3"  # 改成實際的網址

# 發送請求
response = requests.get(url)
response.encoding = 'utf-8'  # 確保中文顯示正常

# 解析 HTML
soup = BeautifulSoup(response.text, 'html.parser')

# 找到所有餐廳區塊
restaurants = soup.find_all("div", class_="B_item_productlist")

headers = {
    "User-Agent": "Mozilla/5.0"
}
requests.get(url, headers=headers)


# 儲存爬取資料
data = []

# 提取每一家餐廳資訊
for r in restaurants:
    try:
        name = r.find("h3", class_="B_item_title").get_text(strip=True)
        address = r.find("span", itemtype="http://schema.org/PostalAddress").get_text(strip=True)
        city_tag = r.find("div", class_="saleIcon green")
        city = city_tag.get_text(strip=True) if city_tag else "未知"

        data.append({
            "餐廳名稱": name,
            "縣市": city,
            "地址": address
        })
    except Exception as e:
        print("發生錯誤：", e)

# 顯示結果
import pandas as pd

df = pd.DataFrame(data)
df.head()


,餐廳名稱,縣市,地址
0,極樂滴水坊,基隆,基隆市信義區信二路270號
1,陶林歡喜素食,基隆,基隆市仁愛區愛一路8號2F(D47攤)
2,田園健康素食,基隆,基隆市安樂區安和一街2巷10號
3,花媽素食,基隆,基隆市中正區義二路159號
4,金庫1985（咖啡.蔬食.創業孵化器）,基隆,基隆市暖暖區暖暖街570巷112號


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 模擬瀏覽器行為
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 基本網址（不含分頁）
base_url = "https://vegemap.merit-times.com/restaurant_list/基隆市-蔬食餐廳"

# 設定要爬幾頁
max_pages = 5

# 儲存所有資料
all_data = []

# 開始爬每一頁
for page in range(1, max_pages + 1):
    # 組合完整 URL
    url = f"{base_url}?page={page}"
    print(f"正在爬第 {page} 頁：{url}")

    # 發送請求
    response = requests.get(url, headers=headers)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    # 抓取所有餐廳項目
    restaurants = soup.find_all("div", class_="B_item_productlist")

    # 萃取每一家餐廳資訊
    for r in restaurants:
        try:
            name = r.find("h3", class_="B_item_title").get_text(strip=True)
            address = r.find("span", itemtype="http://schema.org/PostalAddress").get_text(strip=True)
            city_tag = r.find("div", class_="saleIcon green")
            city = city_tag.get_text(strip=True) if city_tag else "未知"

            all_data.append({
                "餐廳名稱": name,
                "縣市": city,
                "地址": address
            })
        except Exception as e:
            print("⚠️ 發生錯誤：", e)

# 轉成 DataFrame 並顯示
df = pd.DataFrame(all_data)
df.head()
df.shape

正在爬第 1 頁：https://vegemap.merit-times.com/restaurant_list/基隆市-蔬食餐廳?page=1
正在爬第 2 頁：https://vegemap.merit-times.com/restaurant_list/基隆市-蔬食餐廳?page=2
正在爬第 3 頁：https://vegemap.merit-times.com/restaurant_list/基隆市-蔬食餐廳?page=3
正在爬第 4 頁：https://vegemap.merit-times.com/restaurant_list/基隆市-蔬食餐廳?page=4
正在爬第 5 頁：https://vegemap.merit-times.com/restaurant_list/基隆市-蔬食餐廳?page=5


(77, 3)

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 模擬瀏覽器
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 所有縣市列表
cities = [
    "基隆市", "台北市", "新北市", "桃園市", "新竹市", "新竹縣",
    "苗栗縣", "台中市", "彰化縣", "南投縣", "雲林縣", "嘉義市",
    "嘉義縣", "台南市", "高雄市", "屏東縣", "宜蘭縣", "花蓮縣",
    "台東縣", "澎湖縣", "金門縣", "連江縣"
]

# 儲存所有結果
all_data = []

# 開始爬取所有縣市
for city in cities:
    print(f"正在爬取：{city}")
    page = 1
    while True:
        # URL 中文需要處理為 % 編碼
        url = f"https://vegemap.merit-times.com/restaurant_list/{city}-蔬食餐廳?page={page}"
        url = requests.utils.requote_uri(url)

        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        restaurants = soup.find_all("div", class_="B_item_productlist")

        # 如果這一頁沒有餐廳，停止爬這個縣市
        if not restaurants:
            break

        # 提取資料
        for r in restaurants:
            try:
                name = r.find("h3", class_="B_item_title").get_text(strip=True)
                address = r.find("span", itemtype="http://schema.org/PostalAddress").get_text(strip=True)
                city_tag = r.find("div", class_="saleIcon green")
                real_city = city_tag.get_text(strip=True) if city_tag else city

                all_data.append({
                    "餐廳名稱": name,
                    "縣市": real_city,
                    "地址": address
                })
            except Exception as e:
                print(f"⚠️ 錯誤: {e}")

        print(f"✅ {city} - 第 {page} 頁完成，共 {len(restaurants)} 筆")
        page += 1
        time.sleep(1)  # 建議禮貌延遲

# 整理成 DataFrame 並儲存
df = pd.DataFrame(all_data)
df.to_excel("全台蔬食餐廳地址.xlsx", index=False)

# 下載檔案（Colab 用）
from google.colab import files
files.download("全台蔬食餐廳地址.xlsx")


正在爬取：基隆市
✅ 基隆市 - 第 1 頁完成，共 16 筆
✅ 基隆市 - 第 2 頁完成，共 16 筆
✅ 基隆市 - 第 3 頁完成，共 16 筆
✅ 基隆市 - 第 4 頁完成，共 13 筆
✅ 基隆市 - 第 5 頁完成，共 16 筆
✅ 基隆市 - 第 6 頁完成，共 16 筆
✅ 基隆市 - 第 7 頁完成，共 16 筆
✅ 基隆市 - 第 8 頁完成，共 16 筆
✅ 基隆市 - 第 9 頁完成，共 16 筆
✅ 基隆市 - 第 10 頁完成，共 16 筆
✅ 基隆市 - 第 11 頁完成，共 16 筆
✅ 基隆市 - 第 12 頁完成，共 16 筆
✅ 基隆市 - 第 13 頁完成，共 16 筆
✅ 基隆市 - 第 14 頁完成，共 16 筆
✅ 基隆市 - 第 15 頁完成，共 16 筆
✅ 基隆市 - 第 16 頁完成，共 16 筆
✅ 基隆市 - 第 17 頁完成，共 16 筆
✅ 基隆市 - 第 18 頁完成，共 16 筆
✅ 基隆市 - 第 19 頁完成，共 16 筆
✅ 基隆市 - 第 20 頁完成，共 16 筆
✅ 基隆市 - 第 21 頁完成，共 16 筆
✅ 基隆市 - 第 22 頁完成，共 16 筆
✅ 基隆市 - 第 23 頁完成，共 16 筆
✅ 基隆市 - 第 24 頁完成，共 16 筆
✅ 基隆市 - 第 25 頁完成，共 16 筆
✅ 基隆市 - 第 26 頁完成，共 16 筆
✅ 基隆市 - 第 27 頁完成，共 16 筆
✅ 基隆市 - 第 28 頁完成，共 16 筆
✅ 基隆市 - 第 29 頁完成，共 16 筆
✅ 基隆市 - 第 30 頁完成，共 16 筆
✅ 基隆市 - 第 31 頁完成，共 16 筆
✅ 基隆市 - 第 32 頁完成，共 16 筆
✅ 基隆市 - 第 33 頁完成，共 16 筆
✅ 基隆市 - 第 34 頁完成，共 16 筆
✅ 基隆市 - 第 35 頁完成，共 16 筆
✅ 基隆市 - 第 36 頁完成，共 16 筆
✅ 基隆市 - 第 37 頁完成，共 16 筆
✅ 基隆市 - 第 38 頁完成，共 16 筆
✅ 基隆市 - 第 39 頁完成，共 16 筆
✅ 基隆市 - 第 40 頁完成，共 16 筆
✅ 基隆市 - 第 41 頁完成，共 16 筆
✅ 基隆市 - 第 42 頁完成

KeyboardInterrupt: 

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 所有縣市列表（可依實際網站更新）
cities = [
    "基隆市", "台北市", "新北市", "桃園市", "新竹市", "新竹縣", "苗栗縣", "台中市", "彰化縣",
    "南投縣", "雲林縣", "嘉義市", "嘉義縣", "台南市", "高雄市", "屏東縣", "宜蘭縣",
    "花蓮縣", "台東縣", "澎湖縣", "金門縣", "連江縣"
]

# 設定 headers（避免被擋爬蟲）
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 儲存所有資料
all_data = []

# 遍歷每個縣市
for city in cities:
    page = 1
    previous_names = set()

    while True:
        url = f"https://vegemap.merit-times.com/restaurant_list/{city}-蔬食餐廳?page={page}"
        url = requests.utils.requote_uri(url)  # 處理中文網址

        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        # 取得餐廳區塊
        restaurants = soup.find_all("div", class_="B_item_productlist")

        # 若沒有餐廳，停止
        if not restaurants:
            print(f"🚫 {city} - 第 {page} 頁無資料，結束。")
            break

        current_names = set()
        count = 0

        for r in restaurants:
            try:
                name = r.find("h3", class_="B_item_title").get_text(strip=True)
                current_names.add(name)

                address_tag = r.find("span", itemtype="http://schema.org/PostalAddress")
                address = address_tag.get_text(strip=True) if address_tag else "無地址"

                city_tag = r.find("div", class_="saleIcon green")
                real_city = city_tag.get_text(strip=True) if city_tag else city

                all_data.append({
                    "縣市": real_city,
                    "餐廳名稱": name,
                    "地址": address
                })
                count += 1
            except Exception as e:
                print(f"⚠️ 錯誤：{e}")

        # 偵測是否跳針：頁面資料與上一頁相同
        if current_names == previous_names:
            print(f"🛑 {city} - 第 {page} 頁與上一頁重複，停止。")
            break

        previous_names = current_names
        print(f"✅ {city} - 第 {page} 頁完成，共 {count} 筆")
        page += 1
        time.sleep(1)  # 避免過快被擋

# 匯出成 DataFrame 與 Excel/CSV
df = pd.DataFrame(all_data)
df.to_csv("蔬食餐廳總表.csv", index=False, encoding='utf-8-sig')
print("✅ 全部完成，共", len(df), "筆餐廳資料")


✅ 基隆市 - 第 1 頁完成，共 16 筆
✅ 基隆市 - 第 2 頁完成，共 16 筆
✅ 基隆市 - 第 3 頁完成，共 16 筆
✅ 基隆市 - 第 4 頁完成，共 13 筆
✅ 基隆市 - 第 5 頁完成，共 16 筆
🛑 基隆市 - 第 6 頁與上一頁重複，停止。
✅ 台北市 - 第 1 頁完成，共 16 筆
✅ 台北市 - 第 2 頁完成，共 16 筆
✅ 台北市 - 第 3 頁完成，共 16 筆
✅ 台北市 - 第 4 頁完成，共 16 筆
✅ 台北市 - 第 5 頁完成，共 16 筆
✅ 台北市 - 第 6 頁完成，共 16 筆
✅ 台北市 - 第 7 頁完成，共 16 筆
✅ 台北市 - 第 8 頁完成，共 16 筆
✅ 台北市 - 第 9 頁完成，共 16 筆
✅ 台北市 - 第 10 頁完成，共 16 筆
✅ 台北市 - 第 11 頁完成，共 16 筆
✅ 台北市 - 第 12 頁完成，共 16 筆
✅ 台北市 - 第 13 頁完成，共 16 筆
✅ 台北市 - 第 14 頁完成，共 16 筆
✅ 台北市 - 第 15 頁完成，共 16 筆
✅ 台北市 - 第 16 頁完成，共 16 筆
✅ 台北市 - 第 17 頁完成，共 16 筆
✅ 台北市 - 第 18 頁完成，共 16 筆
✅ 台北市 - 第 19 頁完成，共 16 筆
✅ 台北市 - 第 20 頁完成，共 16 筆
✅ 台北市 - 第 21 頁完成，共 16 筆
✅ 台北市 - 第 22 頁完成，共 16 筆
✅ 台北市 - 第 23 頁完成，共 16 筆
✅ 台北市 - 第 24 頁完成，共 16 筆
✅ 台北市 - 第 25 頁完成，共 16 筆
✅ 台北市 - 第 26 頁完成，共 16 筆
✅ 台北市 - 第 27 頁完成，共 16 筆
✅ 台北市 - 第 28 頁完成，共 16 筆
✅ 台北市 - 第 29 頁完成，共 16 筆
✅ 台北市 - 第 30 頁完成，共 16 筆
✅ 台北市 - 第 31 頁完成，共 16 筆
✅ 台北市 - 第 32 頁完成，共 16 筆
✅ 台北市 - 第 33 頁完成，共 16 筆
✅ 台北市 - 第 34 頁完成，共 16 筆
✅ 台北市 - 第 35 頁完成，共 16 筆
✅ 台北市 - 第 36 頁完成，共 16 筆
✅ 台北市 

In [15]:

# 轉換為 DataFrame
df = pd.DataFrame(all_data)

# 移除重複（以餐廳名稱 + 地址判斷）
df = df.drop_duplicates(subset=["餐廳名稱", "地址"], keep="first").reset_index(drop=True)

# 匯出結果
df.to_excel("蔬食餐廳_去重後.xlsx", index=False)
print("✅ 完成！總共", len(df), "筆去重後的餐廳資料。")

✅ 完成！總共 5252 筆去重後的餐廳資料。


In [17]:
# 統計各縣市餐廳數量
city_counts = df.groupby("縣市").size().reset_index(name="餐廳數量")

# 依照餐廳數量由大到小排序
city_counts = city_counts.sort_values(by="餐廳數量", ascending=False).reset_index(drop=True)

# 顯示統計結果
print(city_counts)

# 可選：儲存為 CSV 檔案
city_counts.to_csv("各縣市蔬食餐廳數量統計.csv", index=False, encoding="utf-8-sig")


    縣市  餐廳數量
0   台中   919
1   新北   676
2   台北   669
3   高雄   594
4   台南   492
5   桃園   383
6   彰化   295
7   嘉義   268
8   屏東   257
9   新竹   254
10  南投   235
11  雲林   202
12  花蓮   155
13  宜蘭   148
14  苗栗   103
15  基隆    93
16  台東    85
17  澎湖    52
18  金門    20
19  連江     4


In [16]:
df = pd.DataFrame(all_data)
df.head()
df.shape
#df.to_excel("全台蔬食餐廳地址.xlsx", index=False)

(5904, 3)